In [23]:
import xml.etree.ElementTree as ET 
from xml.etree.cElementTree import ElementTree
from collections import defaultdict
import gzip


Liste des fichiers transit issus de GTFS

In [24]:
companies = {"EXO_chambly":"chamblyrichelieucarignan-20220104",
            "EXO_trains":"exo_trains-20220223",
            "EXO_hautsaintlaurent":"hautsaintlaurent-20211213",
            "EXO_lapresquile":"lapresquile-20220106",
            "EXO_lassomption":"lassomption-20220119",
            "EXO_laurentides":"laurentides-20220406",
            "STL":"laval-20220326",
            "EXO_lerichelain":"lerichelain-20220104",
            "RTL":"longueuil-20180820",
            "STM":"montreal-20220225",
            "EXO_rousillon":"rousillon-20211208",
            "EXO_saintejulie":"saintejulie-20220112",
            "EXO_sorelvarennes":"sorelvarennes-20220321",
            "EXO_sudouest":"sudouest-20211213",
            "EXO_terrebonnemascouche":"terrebonnemascouche-20220107",
            "EXO_valleedurichelieu":"valleedurichelieu-20211215"
            }

Regrouper tous les fichier transit dans le même

In [57]:
vehicleTypeList=[]
vehiclesTree = ET.Element('vehicleDefinitions')
scheduleTree = ET.Element('transitSchedule')
transitStops = ET.SubElement(scheduleTree, 'transitStops')

for company in companies :
    vehiclesFile = ET.parse(gzip.open('output_unmapped/'+companies.get(company)+'/renamed_transit_vehicles.xml.gz','r'))
    vehicles = vehiclesFile.getroot()
    for vehicleType in vehicles.iter('{http://www.matsim.org/files/dtd}vehicleType'):
        if vehicleType.get("id") not in vehicleTypeList :
            vehiclesTree.append(vehicleType)
            vehicleTypeList.append(vehicleType.get("id"))
    for vehicle in vehicles.iter('{http://www.matsim.org/files/dtd}vehicle'):
        vehiclesTree.append(vehicle)
    scheduleFile = ET.parse(gzip.open('output_unmapped/'+companies.get(company)+'/renamed_transit_schedule.xml.gz','r'))
    schedule = scheduleFile.getroot()
    for stop in schedule.iter('stopFacility'):
        transitStops.append(stop)
    for transitLine in schedule.iter('transitLine'):
        scheduleTree.append(transitLine)
    
    

Supprimer les StopFacilities en doublon

In [58]:
from iteration_utilities import duplicates
from iteration_utilities import unique_everseen

stops = []
for stop in transitStops.iter('stopFacility'):
    stops.append(stop.get("id"))

stopsdoublons = list(unique_everseen(duplicates(stops)))

In [59]:
print(stopsdoublons)
print(len(stopsdoublons))

['74529', '74701', '75018', '77507', '77508', '77509', '79118', '71026', '71033', '71034', '71039', '71042', '71047', '71221', '71294', '71296', '71323', '71333', '71356', '76077', '76138', '76557', '77501', '77503', '77504', '77505', '77506', '77655', '77657', '78043', '78045', '78046', '78087', '78088', '78116', '78117', '78123', '78125', '78126', '78132', '78135', '78136', '78161', '78168', '78179', '78180', '78245', '78275', '78315', '78445', '78559', '78563', '78564', '78565', '78567', '78568', '78569', '78606', '78730', '78731', '78734', '78885', '78898', '79191', '82004', '82005', '82007', '82064', '82066', '82067', '82855', '82868', '84125', '84830', '84966', '84967', '85016', '85078', '85086', '85089', '85253', '85776', '70020', '71021', '71022', '71023', '71321', '71332']
88


In [61]:
for stop in stopsdoublons:
    teststops = transitStops.findall(".//*[@id='"+str(stop)+"']")
    for i in range(len(teststops)-1):
        transitStops.remove(teststops[i])

0 74529
1 74529
0 74701
0 75018
0 77507
1 77507
0 77508
1 77508
0 77509
1 77509
0 79118
1 79118
0 71026
0 71033
0 71034
0 71039
0 71042
0 71047
0 71221
0 71294
0 71296
0 71323
0 71333
0 71356
0 76077
0 76138
0 76557
0 77501
0 77503
0 77504
0 77505
0 77506
0 77655
0 77657
0 78043
0 78045
0 78046
0 78087
0 78088
0 78116
0 78117
0 78123
0 78125
0 78126
0 78132
0 78135
0 78136
0 78161
0 78168
0 78179
0 78180
0 78245
0 78275
0 78315
0 78445
0 78559
0 78563
0 78564
0 78565
0 78567
0 78568
0 78569
0 78606
0 78730
0 78731
0 78734
0 78885
0 78898
0 79191
0 82004
0 82005
0 82007
0 82064
0 82066
0 82067
0 82855
0 82868
0 84125
0 84830
0 84966
0 84967
0 85016
0 85078
0 85086
0 85089
0 85253
0 85776
0 70020
0 71021
0 71022
0 71023
0 71321
0 71332


In [62]:
stopsAfter = []
for stop in transitStops.iter('stopFacility'):
    stopsAfter.append(stop.get("id"))

stopsdoublons = list(unique_everseen(duplicates(stopsAfter)))

print(stopsdoublons)
print(len(stopsdoublons))

[]
0


Écriture des fichiers

In [63]:
schedulePath = 'output_unmapped/transit_schedule.xml.gz'
f = gzip.open(schedulePath, 'wb')
f.write('<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE transitSchedule SYSTEM "http://www.matsim.org/files/dtd/transitSchedule_v2.dtd">'.encode('utf8'))
ElementTree(scheduleTree).write(f) 
f.close()

vehiclesPath = 'output_unmapped/transit_vehicles.xml.gz'
g = gzip.open(vehiclesPath, 'wb')
g.write('<?xml version="1.0" encoding="UTF-8"?>'.encode('utf8'))
ElementTree(vehiclesTree).write(g) 
g.close()